In [1]:
import requests
import json
from pprint import pprint
import os
import re

In [2]:
class BitportBlob(object):
    def __init__(self, name):
        self.obj_list = []
        self.auth_tok = ''
        self.me_info = {}
        self.all_bpf_objects = {}
        
    def process_cloud_api_requests_obj(self, the_cloud_api_requests_object):
        xx = jsonify_req_obj(the_cloud_api_requests_object)
        files_dict = xx['data'][0]['folders']
        for file_item in files_dict:
            if file_item['files_count'] > 0:

                temp_obj = BitportFile(file_item['code'])
                
                temp_obj.dictoid = {'name':         file_item['name'],
                                     'size':        file_item['size'], 
                                     'files_count': file_item['files_count'], 
                                     'dir_code':    file_item['code']  # call it dir_code cuz there is a file_code
                                    }
                                                              
                self.obj_list.append(temp_obj)


In [3]:
class BitportFile(object):
    instance_names = []
    fdict = {}
    def __init__(self, name):
        self.name = name
        BitportFile.instance_names.append(self.name)
        
        self.dictoid = {}
        
    def __del__(self):
        print("deling", self)

In [4]:
def get_secrets_from_file(dict1 = {}):
    '''
    Keep secret stuff in a .bitport file rather than embedded in the code.
    This utility reads the file and returns a dict.  (or else creates the file.)
    
    File should look like this (but with real info instead of ????s;)
    {
      "client_id":"?????",
      "client_secret": "??????????",
      "code": "??????"
    } 
    

    '''
    debug = False
    filename = '.bitport'
    if os.path.exists(filename):
        if debug: print ('File {} already exists, reading...'.format(filename))
    
        with open(filename,  'r') as fh:                                                             
            mystuff = json.load(fh)  
        
        if debug: print("FOUND FILE CONTENTS: [[[{}]]]".format(mystuff))

    else:
        print ('File {} not found.   Creating....'.format(filename))

        with open(filename, 'w') as fh:                                                             
            json.dump(words, fh)    
            
    if debug: pprint(payload)

    return mystuff
            
            

In [5]:
def get_auth_tok(url='https://api.bitport.io/v2/oauth2/access-token'):
    '''
    This method does a POST to http://bitport.io/get-access to obtain the 'code' (a.k.a. USER_CODE), 
    per the Bitport doc, "thanks to which you can get access_token" [sic]. 

    Beware that the 'code' seems to change occasionally, but the old one still works. So, it may be that
    a human needs to be logged in to a browser and hit the ./get-access site to get a new one occasionally.

    The requests.post below should yield a blob containing 3 items;
    token_type":"Bearer","expires_in":157766399,"access_token":"eyJ0eXAiOiJKV1QiLCJhbGciO...."
    
    We ony care about, (and therefore only return), the 'access_token' item.  Once we have the access_token
    we can make API calls to do useful stuff on the Bitport site.
    
    For more information see the (very terse) Bitport documentation available here: https://bitport.io/api

    '''
    
    dict1 = get_secrets_from_file('.bitport')

    payload = {'type': 'application/x-www-form-urlencoded', 
               'client_id':'?', 
               'client_secret': '?', 
               'grant_type': 'code', 
               'code': '?'} 
    
    # We read those things from a file for a reason, they are needed in the payload;
    # We keep them out of the distro / git repo.   They are secrets!
    payload['client_id']     = dict1['client_id']
    payload['client_secret'] = dict1['client_secret']
    payload['code']          = dict1['code']


    r = requests.post(url, data=payload)
    if ( r.status_code != 200 ):
        print("\n\nPOST was a FAILURE!!   status_code = : ", r.status_code, "\n\n")
        return False
    else:
        resp = json.loads(r.text)
        return resp['access_token']

  

In [6]:
def jsonify_req_obj(robj):
    import json
    rt = json.loads(robj.text)

    return rt

In [7]:
def bitport_api_cloud(tok):
    '''
    This returns a top level look at your Bitport cloud content.
    '''
    api_url = 'https://api.bitport.io/v2/cloud'

    header = {'Authorization': tok } 

    rr = requests.get(api_url, headers = header)
    return rr

In [8]:
def get_a_files_download_url(tok):
    
    ''' 
    Take the so-called 'fileCode', which looks like this: wzskcf8bls
    use that in the https://api.bitport.io/v2/files/<....>/stream.m3u8 url, and 
    get a result that looks like this;
    
    ['#EXTM3U',
     '#EXT-X-VERSION:4',
     '#EXT-X-TARGETDURATION:3556',
     '#EXT-X-MEDIA-SEQUENCE:0',
     '#EXTINF:3556',
     'https://s04.bitport.io/download/uGDWG96J1QRpEFQnntfzsp49vbkK1oba/1',
     '#EXT-X-ENDLIST']
    
    Then parse out and return just the https (url) piece.
    
    '''
    debug = True
    
    api_url = 'https://api.bitport.io/v2/files/wzskcf8bls/stream.m3u8'

    header = {'Authorization': tok } 

    rr = requests.get(api_url, headers = header)
    
    pat = r'(http.+1)\n'
    import re
    found = re.search(pat, rr.text)
    if debug: print('0=[{}]'.format(found.group(0)))
    if debug: print('1=[{}]'.format(found.group(1)))

    if debug: print('finalanswer[[[{}]]]'.format(found.group(1)[3]))

    return found.group(1)

In [9]:
def get_all_download_urls(bitport_blob_obj):
    
    print (len(bitport_blob_obj.obj_list))
    for jj in bitport_blob_obj.obj_list:

        print('\n\nworking on [{}]'.format(jj.name))
        download_url = get_a_files_download_url(bitport_blob_obj.auth_tok)
        '''
        The download_url should look something like this;
        https://s04.bitport.io/download/uGDWG96J1QRpEFQnntfzsp49vbkK1oba/1
        And undocumented Bitport feature is that there is also a .../2 url
        which provides a smaller (more compressed) MP4 version of the same
        file.   We regex the 1 into a 2 and store that as download_url_sm;
        '''

        download_url_sm = re.sub(r'/1$', '/2', download_url)
        #print("OLD URL = ", download_url)

        #print("NEW URL = ", download_url_sm)

        jj.dictoid['download_url']    = download_url
        jj.dictoid['download_url_sm'] = download_url_sm

        pprint(jj.dictoid)

In [10]:
def get_me_info_from_api(tok):
    '''
    This returns a top level look at your Bitport cloud content.
    '''
    api_url = 'https://api.bitport.io/v2/me'

    header = {'Authorization': tok } 

    rr = requests.get(api_url, headers = header)
    jj = jsonify_req_obj(rr)
    
    return jj['data']

In [11]:
def paint_one_row(row, bpo):
        print('{:>3} {:<12} {:<62} {:>5} {:>12}'.format(
        row,
        bpo.obj_list[row].dictoid['dir_code'], 
        bpo.obj_list[row].dictoid['name'],
        bpo.obj_list[row].dictoid['files_count'],
        bpo.obj_list[row].dictoid['size'])
             )

In [12]:
def paint_one_row_heading(row, bpo):
    print('{:>3} {:<12} {:<62} {:>5} {:>12}'.format(
    row,
    bpo.obj_list[row].dictoid['dir_code'], 
    bpo.obj_list[row].dictoid['name'],
    bpo.obj_list[row].dictoid['files_count'],
    bpo.obj_list[row].dictoid['size'])
         )

In [13]:


def paint_top_folders(bitp_obj):
    
    for ii in range(len(bitp_obj.obj_list)):
        paint_one_row(ii, bitp_obj)
    


In [14]:
def ui(bp_obj):
    print('\n################# Info ##################\n')
    print(bp_obj.me_info)
    print('\n################# Top Folders ##################\n')

    paint_top_folders(bp_obj)
    user_input = input('Pick a number:')
    user_input = int(user_input)
    print('You picked {}\n'.format(user_input))
    paint_one_row(user_input, bp_obj)

In [15]:
def bitport_api_whole_folder(tok):
    api_url = 'https://api.bitport.io/v2/cloud/fbjxvlav3i'
    header = {'Authorization': tok }
    rr = requests.get(api_url, headers = header)
    return rr

In [16]:
def main_guy():
    print("main_guy called...")
    BiBo = BitportBlob('mmm')
    BiBo.auth_tok = get_auth_tok()
    BiBo.me_info = get_me_info_from_api(BiBo.auth_tok)
    BiBo.process_cloud_api_requests_obj(bitport_api_cloud(BiBo.auth_tok))
    ui(BiBo)
    
    thing = bitport_api_whole_folder(BiBo.auth_tok)
    print('type:', type(thing))
    pprint(jsonify_req_obj(thing))
    #dstruct = json.loads(thing)
    print('%' * 80)
    #print('idiot')
    #pprint(dstruct)
    return thing
    

In [17]:
shit = main_guy()

main_guy called...

################# Info ##################

[{'email': 'Mkurtis@gmail.com', 'plan_name': 'Small', 'plan_expiration': {'date': '2017-10-14 08:07:15.000000', 'timezone_type': 3, 'timezone': 'UTC'}, 'disk': {'size': 32212254720, 'available': 5080938506, 'used': 27131316214}}]

################# Top Folders ##################

  0 8yzfuvl00s   Pinky and the Brain                                               70  18250676353
  1 leyun3xjvb   Inception.2010.CAM.XviD-TA                                         7   1522427257
  2 waq2pompk1   Game.of.Thrones.S07E02.720p.HDTV.x264-AVS[rarbg]                   3    973210831
  3 2us5qpavwq   Game.of.Thrones.S07E03.1080p.WEB.h264-TBS[rarbg]                   3   1784365138
  4 40p3set1sc   www.Torrenting.com  - Game.of.Thrones.S07E06.HDTV.x264.AC3-Manning     2   1080345083
  5 nq17okhax4   www.Torrenting.com  - Game.of.Thrones.S07E06.HDTV.x264.AC3-Manning     2   1080345083
Pick a number:0
You picked 0

  0 8yzfuvl00s   Pinky a

In [18]:
type(shit)

requests.models.Response

In [19]:
zz = jsonify_req_obj(shit)
print(type(zz))
pprint(zz)

<class 'dict'>
{'data': None,
 'errors': [{'code': 2404, 'message': 'Folder not found'}],
 'status': 'error'}


In [20]:
dict_head = {}
bag_of_objects = []
for mm in zz['data'][0]['files']:
    #print('9' * 100)
    #pprint(mm.keys())
    #print(mm['name'])
    
    #print(' {:<8} {:<82} {:>5} {:>12}'.format(
    #    1555,
    #    mm['name'],
    #    mm['code'],
    #    mm['size'])
    #)
    dict_head[mm['code']] = {'code': mm['code'], 'name': mm['name'], 'size': mm['size']}
    cc = BitportFile(mm['code'])
    cc.fdict = dict_head
    bag_of_objects.append(cc)

pprint(dict_head)
    
#SSSSSSSSS bag_of_objects


TypeError: 'NoneType' object is not subscriptable

In [21]:
BitportFile.instance_names

['8yzfuvl00s',
 'leyun3xjvb',
 'waq2pompk1',
 '2us5qpavwq',
 '40p3set1sc',
 'nq17okhax4']

In [22]:
for xx in BitportFile.instance_names:
    del xx
    

In [25]:
for crap in bag_of_objects:
    print('%' * 100)
    pprint(crap.fdict)

In [24]:
pprint(shit.all_bpf_objects)

AttributeError: 'Response' object has no attribute 'all_bpf_objects'